## Testing Yu-Gi-Oh API card grabbing

### Import required packages

In [2]:
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize

print("All libraries imported successfully!")

# API source: https://db.ygoprodeck.com/api-guide/

All libraries imported successfully!


In [3]:
# function to add parameters to the API call
def add_parameter(f, v, p):
    if p == "?":
        return "{}{}={}".format(p,f,v)
    else:
        return "&{}{}={}".format(p,f,v)

In [4]:
# function that extracts the category of the venue
def separate_prices(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
params = "?"
url = "https://db.ygoprodeck.com/api/v7/cardinfo.php"

#params = add_parameter("name","Dark Magician",params)
params = add_parameter("archetype","Prank-Kids",params)

if params != "?":
    url = url + params
results = requests.get(url).json()
results
print(url)

https://db.ygoprodeck.com/api/v7/cardinfo.php?archetype=Prank-Kids


In [8]:
cards = results['data']
cards_db = json_normalize(cards)
cards_db.head()

C:\Users\dsalt\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,type,desc,atk,def,level,race,attribute,archetype,card_sets,card_images,card_prices,linkval,linkmarkers
0,81997228,Prank-Kids Battle Butler,Fusion Monster,"""Prank-Kids Lampsies"" + ""Prank-Kids Dropsies"" ...",3000.0,3000.0,10.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81997228, 'image_url': 'https://storag...","[{'cardmarket_price': '0.82', 'tcgplayer_price...",NaN,NaN
1,43886072,Prank-Kids Bow-Wow-Bark,Link Monster,"2 ""Prank-Kids"" monsters\r\nA ""Prank-Kids"" mons...",2000.0,NaN,NaN,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 43886072, 'image_url': 'https://storag...","[{'cardmarket_price': '0.18', 'tcgplayer_price...",2.0,"[Right, Bottom]"
2,17382973,Prank-Kids Dodo-Doodle-Doo,Link Monster,"2 ""Prank-Kids"" monsters\r\nIf this card is Lin...",2000.0,NaN,NaN,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 17382973, 'image_url': 'https://storag...","[{'cardmarket_price': '4.65', 'tcgplayer_price...",2.0,"[Left, Bottom]"
3,55725117,Prank-Kids Dropsies,Effect Monster,If this card is sent to the GY as material for...,1000.0,1000.0,2.0,Aqua,WATER,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 55725117, 'image_url': 'https://storag...","[{'cardmarket_price': '0.31', 'tcgplayer_price...",NaN,NaN
4,81119816,Prank-Kids Fansies,Effect Monster,If this card is sent to the GY as material for...,0.0,2000.0,1.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81119816, 'image_url': 'https://storag...","[{'cardmarket_price': '1.67', 'tcgplayer_price...",NaN,NaN


In [6]:
# Handle card prices and turn into sepearate columns
for item in cards_db["card_prices"][0]:
    for i in item.keys():
        cards_db[i] = item[i]

cards_db.drop(columns='card_prices',inplace=True)
#cards_db['cardmarket_price'] = cards_db['cardmarket_price'].apply(lambda x: x*1.1)  For converting to USD later
cards_db

,id,name,type,desc,atk,def,level,race,attribute,archetype,card_sets,card_images,linkval,linkmarkers,cardmarket_price,tcgplayer_price,ebay_price,amazon_price,coolstuffinc_price
0,81997228,Prank-Kids Battle Butler,Fusion Monster,"""Prank-Kids Lampsies"" + ""Prank-Kids Dropsies"" ...",3000.0,3000.0,10.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81997228, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
1,43886072,Prank-Kids Bow-Wow-Bark,Link Monster,"2 ""Prank-Kids"" monsters\r\nA ""Prank-Kids"" mons...",2000.0,NaN,NaN,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 43886072, 'image_url': 'https://storag...",2.0,"[Right, Bottom]",0.68,0.38,4.25,0.62,0.39
2,17382973,Prank-Kids Dodo-Doodle-Doo,Link Monster,"2 ""Prank-Kids"" monsters\r\nIf this card is Lin...",2000.0,NaN,NaN,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 17382973, 'image_url': 'https://storag...",2.0,"[Left, Bottom]",0.68,0.38,4.25,0.62,0.39
3,55725117,Prank-Kids Dropsies,Effect Monster,If this card is sent to the GY as material for...,1000.0,1000.0,2.0,Aqua,WATER,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 55725117, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
4,81119816,Prank-Kids Fansies,Effect Monster,If this card is sent to the GY as material for...,0.0,2000.0,1.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81119816, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
5,18236002,Prank-Kids Lampsies,Effect Monster,If this card is sent to the GY as material for...,1500.0,500.0,3.0,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 18236002, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
6,79059098,Prank-Kids Pandemonium,Spell Card,"During the Main Phase: Fusion Summon 1 ""Prank-...",NaN,NaN,NaN,Quick-Play,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 79059098, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
7,16269385,Prank-Kids Place,Spell Card,"When this card is activated: You can add 1 ""Pr...",NaN,NaN,NaN,Field,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 16269385, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
8,15447747,Prank-Kids Plan,Trap Card,During the Main Phase: You can activate this e...,NaN,NaN,NaN,Continuous,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 15447747, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
9,43664494,Prank-Kids Pranks,Spell Card,"You can discard 1 ""Prank-Kids"" card; Special S...",NaN,NaN,NaN,Continuous,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 43664494, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39


In [7]:
# Uncomment below line if full text should be displayed
#pd.set_option('display.max_colwidth', -1)
cards_db.head()

,id,name,type,desc,atk,def,level,race,attribute,archetype,card_sets,card_images,linkval,linkmarkers,cardmarket_price,tcgplayer_price,ebay_price,amazon_price,coolstuffinc_price
0,81997228,Prank-Kids Battle Butler,Fusion Monster,"""Prank-Kids Lampsies"" + ""Prank-Kids Dropsies"" ...",3000.0,3000.0,10.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81997228, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
1,43886072,Prank-Kids Bow-Wow-Bark,Link Monster,"2 ""Prank-Kids"" monsters\r\nA ""Prank-Kids"" mons...",2000.0,NaN,NaN,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 43886072, 'image_url': 'https://storag...",2.0,"[Right, Bottom]",0.68,0.38,4.25,0.62,0.39
2,17382973,Prank-Kids Dodo-Doodle-Doo,Link Monster,"2 ""Prank-Kids"" monsters\r\nIf this card is Lin...",2000.0,NaN,NaN,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 17382973, 'image_url': 'https://storag...",2.0,"[Left, Bottom]",0.68,0.38,4.25,0.62,0.39
3,55725117,Prank-Kids Dropsies,Effect Monster,If this card is sent to the GY as material for...,1000.0,1000.0,2.0,Aqua,WATER,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 55725117, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
4,81119816,Prank-Kids Fansies,Effect Monster,If this card is sent to the GY as material for...,0.0,2000.0,1.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81119816, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
